In [26]:
from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import gurobipy as gp
from gurobipy import GRB

In [75]:
def problem_constructor(num_of_clients, arrival_time='uniform', end_time='uniform',
                        util_demand_relation='linear', dim_size=3, time_length=12):
    # arrival_time--uniform,poisson
    # end_time--uniform,gaussian
    # relation--random,quadratic,linear
    size = dim_size + time_length + 4  # demands_per_dim,time_s,time_e,time_interval,utility,chosen or not
    lin_comb = np.random.rand(dim_size) * 0.3 + 0.1
    clients = np.zeros((num_of_clients, size))

    # Create client instances
    for index in range(num_of_clients):
        for dim in range(dim_size):
            # determine demand of the client for each dimension
            clients[index][dim] = rd.uniform(10, 100)
        # start time
        if arrival_time == 'uniform':
            s_time = rd.randint(1, int(0.75 * time_length))  # uniformly random
        else:
            lam = 10
            s_time = np.random.poisson(lam)
            s_time = int(max(1, min(s_time, time_length)))  # starting time poisson
        # end time
        if end_time == 'uniform':
            median = (time_length + s_time) // 2
            e_time = rd.randint(median, time_length + 1)  # uniform endtime
        else:
            mean = s_time + (time_length + 1 - s_time) // 2
            deviation = (time_length + 1 - mean) // 3
            e_time = np.random.normal(mean, deviation)
            e_time = int(max(s_time + 1, min(time_length + 1, e_time)))  # gaussian endtime
        clients[index][dim_size] = s_time
        clients[index][dim_size + 1] = e_time  # end_time excluded
        clients[index][dim_size + 1 + s_time:dim_size + 1 + e_time] = np.ones(e_time - s_time)
        if util_demand_relation == 'linear':
            clients[index][-2] = np.dot(lin_comb, clients[index][:dim_size])  # utility
        elif util_demand_relation == 'quadratic':
            clients[index][-2] = np.dot(lin_comb, clients[index][:dim_size] ** 2)
        else:
            clients[index][-2] = rd.uniform(1, 100)
    time_slot_capacity = np.zeros((dim_size, time_length))
    max_total_demand_per_dim_time = np.matmul(np.transpose(clients[:, :dim_size]),
                                             clients[:, dim_size + 2:dim_size + time_length + 2])
    for dim in range(dim_size):
        time_slot_capacity[dim][0] = rd.uniform(0.21 * max_total_demand_per_dim_time[dim][0],
                                                0.5 * max_total_demand_per_dim_time[dim][0])
        for step in range(1, time_length):
            change = rd.randint(0, 1)
            if change:
                time_slot_capacity[dim][step] = rd.uniform(0.21 * max_total_demand_per_dim_time[dim][step],
                                                           0.5 * max_total_demand_per_dim_time[dim][step])
            else:
                time_slot_capacity[dim][step] = time_slot_capacity[dim][step - 1]
    B_constraint=0.6*time_slot_capacity.max(axis=1).mean()
    ufp_model=GEKKO(remote=False)
    ufp_model.options.SOLVER = 1
    opt_choice = ufp_model.Array(ufp_model.Var, num_of_clients, integer=True, lb=0, ub=1)
    for time in range(1,time_length+1):
        quadr_sum=[[]]*dim_size
        for dim in range(dim_size):
            #linear constraint
            linear_expr = ufp_model.sum([opt_choice[index] * clients[index][dim] * clients[index, dim_size + 1 + time]
                          for index in range(num_of_clients)])
            ufp_model.Equation(linear_expr >= time_slot_capacity[dim, time - 1])
            #coordinates sum for each dimension
            quadr_sum[dim]=ufp_model.sum([opt_choice[index] * clients[index][dim] *clients[index][dim_size + 1 + time] for index in range(num_of_clients)])
        
        quadr_expr=ufp_model.sum([sum**2 for sum in quadr_sum])
        ufp_model.Equation(quadr_expr >= B_constraint**2)
    objective_function = ufp_model.sum([clients[i][-2] * opt_choice[i] for i in range(num_of_clients)])
    ufp_model.Minimize(objective_function)
    ufp_model.options.DIAGLEVEL = 4
    ufp_model.solve(disp=True)


In [71]:
problem_constructor(50)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 Called files( 55 )
 File copy failed: overrides.dbs does not exist
 files: overrides.dbs does not exist
 ARGUMENT  1 : gk_model14
 ARGUMENT  2 : 
 
 Run id   : 2024y03m18d12h10m50.348s
 
 COMMAND LINE ARGUMENTS
 coldstart:  0
 imode    :  3
 dbs_read :  T
 dbs_write:  T
 specs    :  T
 
 rto selected
 Called files( 35 )
 READ info FILE FOR VARIABLE DEFINITION: gk_model14.info
 SS MODEL INIT  0
 Parsing model file gk_model14.apm
 Read model file (sec): 2.3554999999999997
 Initialize constants (sec): 0.
 Determine model size (sec): 1.9023999999999996
 Allocate memory (sec): 0.
 Parse and store model (sec): 1.4844000000000026
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  85
   Constants    :  0
   Variables    :  3869
   Intermediates:  0
   Connections  :  3771


Exception: @error: Solution Not Found
